# Importing libraries

In [1]:
import pandas as pd 
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

dropping columns

In [3]:
data=pd.read_csv('Bengaluru_House_Data.csv')

In [4]:
data.drop(columns=['area_type','availability','society','balcony'],inplace=True)

In [5]:
data.describe()

,bath,price
count,13247.000000,13320.000000
mean,2.692610,112.565627
std,1.341458,148.971674
min,1.000000,8.000000
25%,2.000000,50.000000
50%,2.000000,72.000000
75%,3.000000,120.000000
max,40.000000,3600.000000


In [7]:
data['size']= data['size'].fillna('2 BHK')

In [8]:
data['bath']= data['bath'].fillna(data['bath'].median())

In [9]:
data['bhk']=data['size'].str.split().str.get(0).astype(int)

In [10]:
def convertRange(x):
    temp=x.split('_')
    if len(temp)==2:
        return(float(temp[0])+float(temp[1]))/2
    try:
        return float(x)
    except:
        return None

In [11]:
data['total_sqft']=data['total_sqft'].apply(convertRange)

price per square ft

In [13]:
data['price_per_sqft']=data['price']*100000/data['total_sqft']

In [14]:
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,13073.000000,13320.000000,13320.000000,13320.000000,1.307300e+04
mean,1554.942029,2.688814,112.565627,2.802778,7.949600e+03
std,1238.458773,1.338754,148.971674,1.294496,1.072440e+05
min,1.000000,1.000000,8.000000,1.000000,2.678298e+02
25%,1100.000000,2.000000,50.000000,2.000000,4.265734e+03
50%,1275.000000,2.000000,72.000000,3.000000,5.454545e+03
75%,1670.000000,3.000000,120.000000,3.000000,7.338057e+03
max,52272.000000,40.000000,3600.000000,43.000000,1.200000e+07


In [15]:
data['location']=data['location'].apply(lambda x: x.strip())
location_count=data['location'].value_counts()

In [16]:
location_count_less_10=location_count[location_count<=10]
location_count_less_10

location
BTM 1st Stage                         10
Nagadevanahalli                       10
Basapura                              10
Sector 1 HSR Layout                   10
Dairy Circle                          10
                                      ..
1Channasandra                          1
Hosahalli                              1
Vijayabank bank layout                 1
near Ramanashree California resort     1
Abshot Layout                          1
Name: count, Length: 1054, dtype: int64

In [18]:
data['location']=data['location'].apply(lambda x:'other' if x in location_count_less_10 else x) 

# Outlier detection and removal

In [19]:
data=data[((data['total_sqft']/data['bhk'])>=300)]
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,12329.000000,12329.000000,12329.000000,12329.000000,12329.000000
mean,1590.166773,2.561441,111.444236,2.651472,6322.476758
std,1261.827604,1.072551,152.759322,0.973754,4187.479096
min,300.000000,1.000000,8.440000,1.000000,267.829813
25%,1118.000000,2.000000,49.340000,2.000000,4207.119741
50%,1300.000000,2.000000,70.000000,3.000000,5300.000000
75%,1700.000000,3.000000,115.000000,3.000000,6938.483548
max,52272.000000,16.000000,3600.000000,16.000000,176470.588235


In [22]:
def remove_outlier_sqft(df):
    df_output=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        gen_df=subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_output=pd.concat([df_output,gen_df],ignore_index=True)
    return df_output
data=remove_outlier_sqft(data)
data.describe()

,total_sqft,bath,price,bhk,price_per_sqft
count,10153.000000,10153.000000,10153.000000,10153.000000,10153.000000
mean,1502.725758,2.471880,91.104765,2.573525,5669.621675
std,873.997461,0.971938,86.241338,0.893533,2274.739750
min,300.000000,1.000000,10.000000,1.000000,1250.000000
25%,1110.000000,2.000000,49.000000,2.000000,4242.424242
50%,1283.000000,2.000000,67.000000,2.000000,5183.823529
75%,1650.000000,3.000000,100.000000,3.000000,6451.612903
max,30400.000000,16.000000,2200.000000,16.000000,24509.803922


In [28]:
def bhk_outlier_remover(df):
    exclude_indices=np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats={}
        for bhk , bhk_df in location_df.groupby('bhk'):
            bhk_stats['bhk']={
                'mean':np.mean(bhk_df.price_per_sqft),
                'std':np.std(bhk_df.price_per_sqft),
                'count':bhk_df.shape[0]
            }
        for bhk , bhk_df in location_df.groupby('bhk'):
            stats=bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices,bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [29]:
data=bhk_outlier_remover(data)

In [30]:
data.drop(columns=['size','price_per_sqft'],inplace=True)

# Cleaned data

In [31]:
data.to_csv("Cleaned_data.csv")

In [32]:
X=data.drop(columns=['price'])
y=data['price']

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [38]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [39]:

print(X_test.shape)
print(X_train.shape)

(2031, 4)
(8122, 4)


# Applying Linear Regression


In [41]:
column_trans =make_column_transformer((OneHotEncoder(sparse=False),['location']),remainder='passthrough')

In [42]:
scaler=StandardScaler()

In [45]:
lr=LinearRegression()

In [46]:
pipe=make_pipeline(column_trans,scaler,lr)

In [47]:
pipe.fit(X_train,y_train)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [48]:
y_pred_lr=pipe.predict(X_test)


In [49]:
r2_score(y_test,y_pred_lr)

0.8057092291739887

# Applying LASSO


In [50]:
lasso=Lasso()
pipe=make_pipeline(column_trans,scaler,lasso)
pipe.fit(X_train,y_train)
y_pred_lasso=pipe.predict(X_test)
r2_score(y_test,y_pred_lasso)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0.7971081825428273

# Applying RIDGE

In [51]:
ridge=Ridge()
pipe=make_pipeline(column_trans,scaler,ridge)
pipe.fit(X_train,y_train)

C:\Users\HP\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['location'])])),
                ('standardscaler', StandardScaler()), ('ridge', Ridge())])

In [52]:
y_pred_ridge=pipe.predict(X_test)
r2_score(y_test,y_pred_ridge)

0.8057561111629195

In [54]:
print("No regularization:",r2_score(y_test,y_pred_lr))
print("Lasso:",r2_score(y_test,y_pred_lasso))
print("Ridge: ",r2_score(y_test,y_pred_ridge))

No regularization: 0.8057092291739887
Lasso: 0.7971081825428273
Ridge:  0.8057561111629195


In [55]:
import pickle


In [56]:
pickle.dump(pipe,open('RidgeModel.pkl','wb'))